[View in Colaboratory](https://colab.research.google.com/github/TonyIssacJames/EIP_public/blob/master/DNST_CIFAR10_AUG.ipynb)

In [1]:
# https://keras.io/
#!pip install -q keras
import keras

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 15
num_filter = 24
compression = 0.7
dropout_rate = 0.20

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
import numpy as np

x_train = np.append(x_train, x_train,axis=0)
y_train = np.append(y_train, y_train,axis=0)
print(x_train.shape)
#print(np.append(x_train, x_train,axis=0).shape)
#print(np.append(y_train, y_train,axis=0).shape)
print(y_train.shape)

(100000, 32, 32, 3)
(100000, 10)


In [7]:
from matplotlib import pyplot
from scipy.misc import toimage
def show_imgs(X):
    pyplot.figure(1)
    k = 0
    for i in range(0,4):
        for j in range(0,4):
            pyplot.subplot2grid((4,4),(i,j))
            pyplot.imshow(toimage(X[k]))
            k = k+1
    # show the plot
    pyplot.show()

show_imgs(x_test[:16])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  if __name__ == '__main__':


<Figure size 640x480 with 16 Axes>

In [8]:
print(y_test[:16])

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [9]:
#Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,    # randomly flip images
        fill_mode='nearest')

In [10]:
#Callback for saving the best model
from keras.callbacks import ModelCheckpoint
filepath= "weights_tr_05_v2.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [11]:
# Dense Block
def add_denseblock(input, num_filter = 24, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [12]:
def add_transition(input, num_filter = 24, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [13]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [14]:
#num_filter = 16
#Sdropout_rate = 0.2
#l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Fourth_Block = add_denseblock(Third_Transition, num_filter, dropout_rate)
Fourth_Transition = add_transition(Fourth_Block, num_filter, dropout_rate)

#Fifth_Block = add_denseblock(Fourth_Transition, num_filter, dropout_rate)
#Fifth_Transition = add_transition(Fifth_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Fourth_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [15]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 24)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [16]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [17]:
#Save the data so reloading will be easy
#np.save("x_train", x_train)
#np.save("y_train", y_train)
#np.save("x_test", x_test)
#np.save("y_test", y_test)


In [27]:
'''model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)'''

model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Epoch 1/5
782/782 [==============================] - 764s 977ms/step - loss: 1.6354 - acc: 0.3932 - val_loss: 1.9286 - val_acc: 0.4274

Epoch 00001: val_acc improved from -inf to 0.42740, saving model to weights_tr_05_v2.best.hdf5
Epoch 2/5
782/782 [==============================] - 671s 858ms/step - loss: 1.2677 - acc: 0.5436 - val_loss: 2.4901 - val_acc: 0.4513

Epoch 00002: val_acc improved from 0.42740 to 0.45130, saving model to weights_tr_05_v2.best.hdf5
Epoch 3/5
782/782 [==============================] - 671s 858ms/step - loss: 0.9840 - acc: 0.6521 - val_loss: 1.0411 - val_acc: 0.6595

Epoch 00004: val_acc improved from 0.56930 to 0.65950, saving model to weights_tr_05_v2.best.hdf5
Epoch 5/5
782/782 [==============================] - 670s 856ms/step - loss: 0.9076 - acc: 0.6830 - val_loss: 1.7530 - val_acc: 0.5659

Epoch 00005: val_acc did not improve from 0.65950


In [28]:
model.save('weights_tr_05_v2_epoch_2.hdf5')  # creates a HDF5 file 'my_model.h5'

In [18]:
#Save the data so reloading will be easy
#np.save("x_train", x_train)
#np.save("y_train", y_train)
#np.save("x_test", x_test)
#np.save("y_test", y_test)

In [30]:
'''model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)'''

model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Epoch 1/5
782/782 [==============================] - 668s 854ms/step - loss: 0.8411 - acc: 0.7075 - val_loss: 0.9860 - val_acc: 0.7146

Epoch 00001: val_acc improved from 0.65950 to 0.71460, saving model to weights_tr_05_v2.best.hdf5
Epoch 2/5
782/782 [==============================] - 671s 858ms/step - loss: 0.7908 - acc: 0.7256 - val_loss: 1.5897 - val_acc: 0.5887

Epoch 00002: val_acc did not improve from 0.71460
Epoch 3/5
782/782 [==============================] - 672s 859ms/step - loss: 0.7488 - acc: 0.7417 - val_loss: 1.0648 - val_acc: 0.6928

Epoch 00003: val_acc did not improve from 0.71460
Epoch 4/5
782/782 [==============================] - 668s 855ms/step - loss: 0.7123 - acc: 0.7544 - val_loss: 0.9650 - val_acc: 0.7041

Epoch 00004: val_acc did not improve from 0.71460
Epoch 5/5
782/782 [==============================] - 667s 853ms/step - loss: 0.6820 - acc: 0.7659 - val_loss: 0.7442 - val_acc: 0.7739

Epoch 00005: val_acc improved from 0.71460 to 0.77390, saving model to w

In [31]:
model.save('weights_tr_05_v2_epoch_10.hdf5')  # creates a HDF5 file 'my_model.h5'

In [19]:
#Save the data so reloading will be easy
#np.save("x_train", x_train)
#np.save("y_train", y_train)
#np.save("x_test", x_test)
#np.save("y_test", y_test)

In [33]:
'''model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)'''

model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Epoch 1/5
782/782 [==============================] - 672s 859ms/step - loss: 0.6521 - acc: 0.7763 - val_loss: 0.8179 - val_acc: 0.7610

Epoch 00001: val_acc did not improve from 0.77390
Epoch 2/5
782/782 [==============================] - 671s 859ms/step - loss: 0.6295 - acc: 0.7833 - val_loss: 0.8100 - val_acc: 0.7533

Epoch 00002: val_acc did not improve from 0.77390
Epoch 3/5
782/782 [==============================] - 672s 859ms/step - loss: 0.6029 - acc: 0.7932 - val_loss: 0.8990 - val_acc: 0.7452

Epoch 00003: val_acc did not improve from 0.77390
Epoch 4/5
782/782 [==============================] - 672s 859ms/step - loss: 0.5842 - acc: 0.7987 - val_loss: 0.8529 - val_acc: 0.7665

Epoch 00004: val_acc did not improve from 0.77390
Epoch 5/5
782/782 [==============================] - 671s 858ms/step - loss: 0.5647 - acc: 0.8047 - val_loss: 0.7705 - val_acc: 0.7722

Epoch 00005: val_acc did not improve from 0.77390


In [35]:
model.save('weights_tr_05_v2_epoch_15.hdf5')  # creates a HDF5 file 'my_model.h5'

In [36]:
'''model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)'''

model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Epoch 1/5
782/782 [==============================] - 671s 858ms/step - loss: 0.5484 - acc: 0.8109 - val_loss: 0.7213 - val_acc: 0.7942

Epoch 00001: val_acc improved from 0.77390 to 0.79420, saving model to weights_tr_05_v2.best.hdf5
Epoch 2/5
782/782 [==============================] - 671s 858ms/step - loss: 0.5333 - acc: 0.8172 - val_loss: 0.6789 - val_acc: 0.8038

Epoch 00002: val_acc improved from 0.79420 to 0.80380, saving model to weights_tr_05_v2.best.hdf5
Epoch 3/5
782/782 [==============================] - 671s 858ms/step - loss: 0.5172 - acc: 0.8214 - val_loss: 0.7315 - val_acc: 0.7902

Epoch 00003: val_acc did not improve from 0.80380
Epoch 4/5
782/782 [==============================] - 846s 1s/step - loss: 0.5077 - acc: 0.8251 - val_loss: 0.7059 - val_acc: 0.7973

Epoch 00004: val_acc did not improve from 0.80380
Epoch 5/5
782/782 [==============================] - 1217s 2s/step - loss: 0.4963 - acc: 0.8289 - val_loss: 0.7094 - val_acc: 0.8016

Epoch 00005: val_acc did not 

In [37]:
model.save('weights_tr_05_v2_epoch_20.hdf5')  # creates a HDF5 file 'my_model.h5'

In [19]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 18s 2ms/step
Test loss: 0.5077479893147946
Test accuracy: 0.8528


In [38]:
'''model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)'''

model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Epoch 1/5
782/782 [==============================] - 1234s 2s/step - loss: 0.4872 - acc: 0.8324 - val_loss: 0.7691 - val_acc: 0.7863

Epoch 00001: val_acc did not improve from 0.80380
Epoch 2/5
782/782 [==============================] - 1235s 2s/step - loss: 0.4744 - acc: 0.8374 - val_loss: 0.8123 - val_acc: 0.7831

Epoch 00002: val_acc did not improve from 0.80380
Epoch 3/5
782/782 [==============================] - 1237s 2s/step - loss: 0.4653 - acc: 0.8391 - val_loss: 0.6871 - val_acc: 0.8062

Epoch 00003: val_acc improved from 0.80380 to 0.80620, saving model to weights_tr_05_v2.best.hdf5
Epoch 4/5
782/782 [==============================] - 1235s 2s/step - loss: 0.4590 - acc: 0.8421 - val_loss: 0.8698 - val_acc: 0.7767

Epoch 00004: val_acc did not improve from 0.80620
Epoch 5/5
782/782 [==============================] - 1236s 2s/step - loss: 0.4491 - acc: 0.8462 - val_loss: 0.7899 - val_acc: 0.7944

Epoch 00005: val_acc did not improve from 0.80620


In [39]:
model.save('weights_tr_05_v2_epoch_25.hdf5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
#continue from here on the prviously saved model after 25 epochs

In [20]:
#Callback for Cyclic LR
#from clr_callback import *
#clr = CyclicLR(base_lr=0.0001, max_lr=0.006, step_size=2000., mode='triangular2')
#callbacks_list = [checkpoint, clr]

In [21]:
from keras.models import load_model

model = load_model('weights_tr_05_v2_epoch_25.hdf5')
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 43s 4ms/step
Test loss: 0.7898893457889556
Test accuracy: 0.7944


In [23]:
'''model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)'''

model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Epoch 1/5
782/782 [==============================] - 383s 490ms/step - loss: 0.4390 - acc: 0.8489 - val_loss: 0.6460 - val_acc: 0.8196

Epoch 00001: val_acc improved from -inf to 0.81960, saving model to weights_tr_05_v2.best.hdf5
Epoch 2/5
782/782 [==============================] - 364s 465ms/step - loss: 0.4308 - acc: 0.8511 - val_loss: 0.8679 - val_acc: 0.7722

Epoch 00002: val_acc did not improve from 0.81960
Epoch 3/5
782/782 [==============================] - 251s 321ms/step - loss: 0.4233 - acc: 0.8542 - val_loss: 0.5758 - val_acc: 0.8335

Epoch 00003: val_acc improved from 0.81960 to 0.83350, saving model to weights_tr_05_v2.best.hdf5
Epoch 4/5
782/782 [==============================] - 181s 231ms/step - loss: 0.4153 - acc: 0.8561 - val_loss: 0.6232 - val_acc: 0.8196

Epoch 00004: val_acc did not improve from 0.83350
Epoch 5/5
782/782 [==============================] - 183s 234ms/step - loss: 0.4100 - acc: 0.8580 - val_loss: 0.6012 - val_acc: 0.8279

Epoch 00005: val_acc did no

In [25]:
model.save('weights_tr_05_v2_01_epoch_30.hdf5')  # creates a HDF5 file 'my_model.h5'

In [26]:
'''model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)'''

model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Epoch 1/5
782/782 [==============================] - 181s 232ms/step - loss: 0.4032 - acc: 0.8613 - val_loss: 0.8183 - val_acc: 0.7862

Epoch 00001: val_acc did not improve from 0.83350
Epoch 2/5
782/782 [==============================] - 181s 232ms/step - loss: 0.3992 - acc: 0.8616 - val_loss: 0.4627 - val_acc: 0.8580

Epoch 00002: val_acc improved from 0.83350 to 0.85800, saving model to weights_tr_05_v2.best.hdf5
Epoch 3/5
782/782 [==============================] - 181s 232ms/step - loss: 0.3928 - acc: 0.8650 - val_loss: 0.5665 - val_acc: 0.8408

Epoch 00003: val_acc did not improve from 0.85800
Epoch 4/5
782/782 [==============================] - 183s 234ms/step - loss: 0.3878 - acc: 0.8667 - val_loss: 0.5736 - val_acc: 0.8409

Epoch 00004: val_acc did not improve from 0.85800
Epoch 5/5
782/782 [==============================] - 181s 232ms/step - loss: 0.3810 - acc: 0.8674 - val_loss: 0.5771 - val_acc: 0.8405

Epoch 00005: val_acc did not improve from 0.85800


In [27]:
model.save('weights_tr_05_v2_01_epoch_35.hdf5')  # creates a HDF5 file 'my_model.h5'

In [28]:
'''model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)'''

model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Epoch 1/5
782/782 [==============================] - 181s 232ms/step - loss: 0.3725 - acc: 0.8713 - val_loss: 0.5250 - val_acc: 0.8460

Epoch 00001: val_acc did not improve from 0.85800
Epoch 2/5
782/782 [==============================] - 181s 232ms/step - loss: 0.3718 - acc: 0.8723 - val_loss: 0.5498 - val_acc: 0.8434

Epoch 00002: val_acc did not improve from 0.85800
Epoch 3/5
782/782 [==============================] - 181s 232ms/step - loss: 0.3642 - acc: 0.8733 - val_loss: 0.4524 - val_acc: 0.8660

Epoch 00003: val_acc improved from 0.85800 to 0.86600, saving model to weights_tr_05_v2.best.hdf5
Epoch 4/5
782/782 [==============================] - 182s 233ms/step - loss: 0.3629 - acc: 0.8746 - val_loss: 0.4293 - val_acc: 0.8760

Epoch 00004: val_acc improved from 0.86600 to 0.87600, saving model to weights_tr_05_v2.best.hdf5
Epoch 5/5
782/782 [==============================] - 183s 235ms/step - loss: 0.3592 - acc: 0.8757 - val_loss: 0.4635 - val_acc: 0.8668

Epoch 00005: val_acc did

In [30]:
model.save('weights_tr_05_v2_01_epoch_40.hdf5')  # creates a HDF5 file 'my_model.h5'

In [31]:
'''model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)'''

model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Epoch 1/5
782/782 [==============================] - 185s 236ms/step - loss: 0.3555 - acc: 0.8768 - val_loss: 0.5389 - val_acc: 0.8429

Epoch 00001: val_acc did not improve from 0.87600
Epoch 2/5
782/782 [==============================] - 183s 235ms/step - loss: 0.3500 - acc: 0.8782 - val_loss: 0.4083 - val_acc: 0.8794

Epoch 00002: val_acc improved from 0.87600 to 0.87940, saving model to weights_tr_05_v2.best.hdf5
Epoch 3/5
782/782 [==============================] - 181s 232ms/step - loss: 0.3479 - acc: 0.8801 - val_loss: 0.5198 - val_acc: 0.8556

Epoch 00003: val_acc did not improve from 0.87940
Epoch 4/5
782/782 [==============================] - 182s 233ms/step - loss: 0.3411 - acc: 0.8829 - val_loss: 0.6450 - val_acc: 0.8343

Epoch 00004: val_acc did not improve from 0.87940
Epoch 5/5
782/782 [==============================] - 183s 235ms/step - loss: 0.3374 - acc: 0.8833 - val_loss: 0.6072 - val_acc: 0.8423

Epoch 00005: val_acc did not improve from 0.87940


In [32]:
model.save('weights_tr_05_v2_01_epoch_45.hdf5')  # creates a HDF5 file 'my_model.h5'

In [33]:
'''model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)'''

model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Epoch 1/5
782/782 [==============================] - 183s 235ms/step - loss: 0.3367 - acc: 0.8831 - val_loss: 0.3829 - val_acc: 0.8876

Epoch 00001: val_acc improved from 0.87940 to 0.88760, saving model to weights_tr_05_v2.best.hdf5
Epoch 2/5
782/782 [==============================] - 183s 233ms/step - loss: 0.3306 - acc: 0.8846 - val_loss: 0.5001 - val_acc: 0.8606

Epoch 00002: val_acc did not improve from 0.88760
Epoch 3/5
782/782 [==============================] - 182s 233ms/step - loss: 0.3308 - acc: 0.8855 - val_loss: 0.5275 - val_acc: 0.8555

Epoch 00003: val_acc did not improve from 0.88760
Epoch 4/5
782/782 [==============================] - 181s 231ms/step - loss: 0.3246 - acc: 0.8857 - val_loss: 0.6510 - val_acc: 0.8335

Epoch 00004: val_acc did not improve from 0.88760
Epoch 5/5
782/782 [==============================] - 181s 232ms/step - loss: 0.3216 - acc: 0.8885 - val_loss: 0.5107 - val_acc: 0.8556

Epoch 00005: val_acc did not improve from 0.88760


In [34]:
model.save('weights_tr_05_v2_01_epoch_50.hdf5')  # creates a HDF5 file 'my_model.h5'

In [35]:
from keras.models import load_model

new_model = load_model("weights_tr_05_v2.best.hdf5")
score = new_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 14s 1ms/step
Test loss: 0.38292595955729486
Test accuracy: 0.8876


In [0]:
#from google.colab import files

#files.download('DNST_model.h5')